In [1]:
# Evaluation data

import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [2]:
# For evaluating retrieval

from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# Q1. Minsearch text

Now let's evaluate our usual minsearch approach, indexing documents with:
```python
text_fields=["question", "section", "text"],
keyword_fields=["course", "id"]
```
but tweak the parameters for search. Let's use the following boosting params:

```python
boost = {'question': 1.5, 'section': 0.1}
```

What's the hitrate for this approach?

* 0.64
* 0.74
* 0.84
* 0.94

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)
index.fit(documents)


def min_search(query, course):
    boost = {'question': 1.5, 'section': 0.1}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )
    return results


evaluate(ground_truth, lambda q: min_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.848714069591528, 'mrr': 0.7288235717887772}

## Embeddings 

The latest version of minsearch also supports vector search. 
We will use it:

```python
from minsearch import VectorSearch
```

We will also use TF-IDF and Singular Value Decomposition to 
create embeddings from texts. You can refer to our
["Create Your Own Search Engine" workshop](https://github.com/alexeygrigorev/build-your-own-search-engine)
if you want to know more about it.

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
```

Let's create embeddings for the "question" field:

```python
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)
```

In [ ]:
from minsearch import VectorSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

# Q2. Vector search for question

Now let's index these embeddings with minsearch:

```python
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)
```

Evaluate this seach method. What's MRR for it?

- 0.25
- 0.35
- 0.45
- 0.55

In [5]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

def vector_search(query, course):
    X_query = pipeline.transform([query])
    results = vindex.search(
        query_vector=X_query,
        filter_dict={'course': course},
        num_results=5
    )
    return results


evaluate(ground_truth, lambda q: vector_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.48195374972984656, 'mrr': 0.35720048987825087}

# Q3. Vector search for question and answer

We only used question in Q2. We can use both question and answer:

```python
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)
```

Using the same pipeline (`min_df=3 for TF-IDF vectorizer and `n_components=128` for SVD), evaluate the performance of this
approach

What's the hitrate?

- 0.62
- 0.72
- 0.82
- 0.92

In [6]:
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)

X = pipeline.fit_transform(texts)

vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

evaluate(ground_truth, lambda q: vector_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.8210503566025502, 'mrr': 0.6717347453353508}

# Q4. Qdrant

Now let's evaluate the following settings in Qdrant:

- `text = doc['question'] + ' ' + doc['text']`
- `model_handle = "jinaai/jina-embeddings-v2-small-en"`
- `limit = 5`

What's the MRR?

- 0.65
- 0.75
- 0.85
- 0.95

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [11]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models


client = QdrantClient("http://localhost:6333")

EMBEDDING_DIMENSIONALITY = 512

collection_name = "hw3"
model_handle = "jinaai/jina-embeddings-v2-small-en"

# Delete the collection if exists
client.delete_collection(collection_name=collection_name)

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)


points = []
id = 0

for doc in documents:
    text = doc['question'] + ' ' + doc['text']

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_handle),
        payload={
            "id": doc['id'],
            "text": text,
            "section": doc['section'],
            "course": doc['course']
        }
    )
    points.append(point)

    id += 1

client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
def qdrant_search(query, course):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=5,  # top closest matches
        with_payload=True,  # to get metadata in the results
        query_filter=models.Filter(must=[
            models.FieldCondition(
                key="course",
                match=models.MatchValue(value=course)
            )
        ])
    )

    return [hit.payload for hit in results.points]


evaluate(ground_truth, lambda q: qdrant_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9299762264966501, 'mrr': 0.8517722066133576}

# Q5. Cosine simiarity

In the second part of the module, we looked at evaluating
the entire RAG approach. In particular, we looked at 
comparing the answer generated by our system with the actual
answer from the FAQ.

One of the ways of doing it is using the cosine similarity. 
Let's see how to calculate it.

Cosine similarity is a dot product between two normalized vectors.
In geometrical sense, it's the cosine of the angle between
the vectors. Look up "cosine similarity geometry" if you want to
learn more about it.

For us, it means that we need two things:

- First, we normalize each of the vectors
- Then, compute the dot product

So, we get this:

```python
def cosine(u, v):
    u = normalize(u)
    v = normalize(v)
    return u.dot(v)
```

For normalization, we first compute the vector norm (its length),
and then divide the vector by it:

```python
def normalize(u):
    norm = np.sqrt(u.dot(u))
    return u / norm
```

(where `np` is `import numpy as np`)

Or we can simplify it:

```python
def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)
```

Now let's use this function to compute the
A->Q->A cosine similarity.

We will use the results from [our gpt-4o-mini evaluations](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-evaluation/rag_evaluation/data/results-gpt4o-mini.csv):


```python
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)
```


When creating embeddings, we will use a simple way -
the same we used in the [Embeddings](#embeddings) section:

```python
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
```

Let's fit the vectorizer on all the text data we have:

```python
pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)
```

Now use the `transform` method of the pipeline to create the embeddings and calculate the cosine similarity between each
pair.

What's the average cosine?

- 0.64
- 0.74
- 0.84
- 0.94

This is how you do it:

- For each answer pair, compute
    - `v_llm` for the answer from the LLM 
    - `v_orig` for the original answer
    - then compute the cosine between them
- At the end, take the average


In [19]:
import numpy as np

def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)

v_llm = pipeline.transform(df_results["answer_llm"])
v_orig = pipeline.transform(df_results["answer_orig"])

similarities = []
for i in range(len(df_results)):
    similarities.append(cosine(v_llm[i], v_orig[i]))

np.mean(similarities)

np.float64(0.8415841233490402)

# Q6. Rouge

And alternative way to see how two texts are similar is ROUGE. 

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```bash
pip install rouge
```

(The latest version at the moment of writing is `1.0.1`)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()

r = df_results.iloc[10]
scores = rouge_scorer.get_scores(r.answer_llm, r.answer_orig)[0]
scores
```

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

For the 10th document, Rouge-1 F1 score is 0.45

Let's compute it for the pairs in the entire dataframe.
What's the average Rouge-1 F1?

- 0.25
- 0.35
- 0.45
- 0.55

In [25]:
from rouge import Rouge

rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df_results["answer_llm"], df_results["answer_orig"])

rouge_1, rouge_2, rouge_l = [], [], []

for i in range(len(scores)):
    rouge_1.append(scores[i]['rouge-1']['f'])
    rouge_2.append(scores[i]['rouge-2']['f'])
    rouge_l.append(scores[i]['rouge-l']['f'])

np.mean(rouge_1), np.mean(rouge_2), np.mean(rouge_l)

(np.float64(0.3516946452113943),
 np.float64(0.17671704698262222),
 np.float64(0.32758565643306675))